In [1]:
import sidewalkify as swky
import osmnx as ox
import numpy as np
import pandas as pd
import networkx as nx
import shapely as shapely
import geopandas as gpd

In [2]:
from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()

<ipython-input-2-7721cd5e3608>:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`
  from tqdm._tqdm_notebook import tqdm_notebook
/home/adam/.local/lib/python3.8/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


# Get the initial road map
Here's what osmnx thinks of as "walking":

In [3]:
ox.downloader._get_osm_filter("walk")

'["area"!~"yes"]["highway"!~"cycleway|motor|proposed|construction|abandoned|platform|raceway"]["foot"!~"no"]["service"!~"private"]["access"!~"private"]'

See EDA for our `cf` choices:

In [4]:
cf = '["highway"~"trunk|trunk_link|tertiary|tertiary_link|secondary|residential|primary|primary_link"]'
G_roads_init = ox.graph.graph_from_place("Boston, MA", network_type="walk", custom_filter=cf)


In [8]:
gdf_roads_init = ox.utils_graph.graph_to_gdfs(G_roads_init)


# Sidewalkify!

## Get into form

Try the easiest thing:

In [9]:
G_roads_init_proj = ox.project_graph(G_roads_init)

In [10]:
gdf_roads_init = ox.utils_graph.graph_to_gdfs(G_roads_init_proj)
gdf_roads_init[1]['sw_left'] = 1
gdf_roads_init[1]['sw_right'] = 1

gdf = gdf_roads_init[1]
gdf = gdf.rename(columns={'osmid':'id'})
gdf


,u,v,key,id,oneway,lanes,name,highway,width,length,...,maxspeed,ref,junction,bridge,tunnel,access,service,area,sw_left,sw_right
0,61341696,61341267,0,8652528,False,2,Park Plaza,secondary,29.9,44.954,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1
1,61341696,61341694,0,8651073,False,2,Park Plaza,secondary,21.3,98.760,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1
2,61341696,61365153,0,426831017,False,2,Columbus Avenue,residential,23.8,26.212,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1
3,61341697,61342333,0,8640801,False,1,Glenville Avenue,residential,12.2,23.799,...,25 mph,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1
4,61341697,61341707,0,8640801,False,1,Glenville Avenue,residential,12.2,72.236,...,25 mph,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30335,61341693,61342336,0,591284846,False,2,Faneuil Street,tertiary,14.3,37.354,...,25 mph,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1
30336,61341693,61341816,0,591284848,False,2,Brooks Street,tertiary,NaN,61.641,...,25 mph,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1
30337,61341694,61341696,0,8651073,False,2,Park Plaza,secondary,21.3,98.760,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1
30338,61341694,61366246,0,242652626,False,2,Charles Street South,primary,30.2,81.007,...,NaN,MA 28,NaN,NaN,NaN,NaN,NaN,NaN,1,1


In [11]:
def fix_list_entries(x):
    if type(x) == list:
        return x[0]
    else:
        return x
gdf2 = gdf.copy().drop(columns=['width','lanes', 'maxspeed','ref','bridge','access','junction','tunnel','service','area', 'oneway'])

gdf2.highway = gdf2.highway.map(fix_list_entries)
gdf2.id = gdf2.id.map(fix_list_entries)
gdf2.name = gdf2.name.map(fix_list_entries)
gdf2.to_file("boston_streets_init_1", driver="GeoJSON")
gdf2.to_file("boston_streets_init_2")

In [17]:
gdf2

,u,v,key,id,name,highway,length,geometry,sw_left,sw_right
0,61341696,61341267,0,8652528,Park Plaza,secondary,44.954,"LINESTRING (329642.117 4690896.342, 329598.118...",1,1
1,61341696,61341694,0,8651073,Park Plaza,secondary,98.760,"LINESTRING (329642.117 4690896.342, 329649.142...",1,1
2,61341696,61365153,0,426831017,Columbus Avenue,residential,26.212,"LINESTRING (329642.117 4690896.342, 329643.924...",1,1
3,61341697,61342333,0,8640801,Glenville Avenue,residential,23.799,"LINESTRING (324297.700 4690875.726, 324277.836...",1,1
4,61341697,61341707,0,8640801,Glenville Avenue,residential,72.236,"LINESTRING (324297.700 4690875.726, 324324.209...",1,1
...,...,...,...,...,...,...,...,...,...,...
30335,61341693,61342336,0,591284846,Faneuil Street,tertiary,37.354,"LINESTRING (321838.540 4691170.987, 321801.767...",1,1
30336,61341693,61341816,0,591284848,Brooks Street,tertiary,61.641,"LINESTRING (321838.540 4691170.987, 321845.783...",1,1
30337,61341694,61341696,0,8651073,Park Plaza,secondary,98.760,"LINESTRING (329739.690 4690908.346, 329734.232...",1,1
30338,61341694,61366246,0,242652626,Charles Street South,primary,81.007,"LINESTRING (329739.690 4690908.346, 329727.484...",1,1


In [9]:
for column in ['maxspeed',
       'ref', 'junction', 'bridge', 'tunnel', 'access', 'service', 'area',
       'sw_left', 'sw_right']:
    print(column)
    print(len(gdf2[gdf2[column].map(type) == list]))

maxspeed


NameError: name 'gdf2' is not defined

In [ ]:
list(list(gdf2.values)[25][6].coords)

In [ ]:
sidewalk_graph_init = swky.graph.create_graph(gdf)
sidewalk_graph_init.edges

In [ ]:
[((a,b),(c,d)) for ((a,b),(c,d)) in sidewalk_graph_init.edges if a is None or b is None or c is None or d is None]

In [ ]:
paths = swky.graph.graph_workflow(gdf)

In [ ]:
drawing = swky.draw.draw_sidewalks(paths, crs=gdf.crs)

In [ ]:
# need to add points to gdf_roads_init[0]!
# can start indexing from 0
points = drawing.geometry
points = points.map(lambda x : x.coords).explode()

In [ ]:
xs = list(points.map(lambda x : x[0]))
ys = list(points.map(lambda x : x[1]))
df = pd.DataFrame({'y':ys, 'x':xs, 'id':range(len(xs))})

In [ ]:
gdf_new_pts = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(xs,ys))
gdf_new_pts['highway'] = np.NaN

In [ ]:
gdf_new_pts

In [ ]:
gdf_new_pts.crs="EPSG:26918"
gdf_new_pts

In [ ]:
gdf_roads_init[0]

In [ ]:
all_pts = gpd.GeoDataFrame(pd.concat([gdf_new_pts.rename(columns={'id':'osmid'}), gdf_roads_init[0]], ignore_index=True,sort=True), crs=gdf_new_pts.crs)

In [ ]:
all_pts_series = all_pts.drop(columns=['highway','lat','lon','x','y']).set_index('osmid')
all_coords_series = all_pts_series.geometry.map(lambda x : tuple(x.coords))


In [ ]:
inv_all_coords = dict((v,k) for k,v in all_coords_series.iteritems())


In [ ]:
drawing['points'] = drawing.geometry.map(lambda x : x.coords).map(lambda ps : [ps[0],ps[1]])

In [ ]:
drawing['u'] = drawing['points'].map(lambda x : x[0])
drawing['v'] = drawing['points'].map(lambda x : x[1])

In [ ]:
drawing['u_x'] = drawing['u'].map(lambda p : p[0])
drawing['v_x'] = drawing['v'].map(lambda p : p[0])
drawing['u_y'] = drawing['u'].map(lambda p : p[1])
drawing['v_y'] = drawing['v'].map(lambda p : p[1])

In [ ]:
drawing['u'] = gpd.points_from_xy(drawing['u_x'],drawing['u_y'])
drawing['v'] = gpd.points_from_xy(drawing['v_x'],drawing['v_y'])

In [ ]:
drawing = drawing.drop(columns=['u_x','u_y','v_x','v_y'])

In [ ]:
drawing['u'] = drawing['u'].map(lambda x : inv_all_coords[tuple(x.coords)])
drawing['v'] = drawing['v'].map(lambda x : inv_all_coords[tuple(x.coords)])

In [ ]:
drawing = drawing.drop(columns='points')


In [ ]:
# 1 for sidewalk?
drawing['key'] = 1

In [ ]:
all_pts.crs

In [ ]:
print(drawing.geometry)
print(gdf.geometry)
print(gdf.crs)

In [ ]:
streets_and_sidewalks = gpd.GeoDataFrame(pd.concat([drawing,gdf], ignore_index=True, sort=False), crs=gdf.crs)
just_sidewalks = gpd.GeoDataFrame(drawing, crs=gdf.crs)

streets_and_sidewalks['osmid'] = streets_and_sidewalks['id']

streets_and_sidewalks = streets_and_sidewalks.to_crs("epsg:4326")
just_sidewalks = just_sidewalks.to_crs("epsg:4326")

streets_and_sidewalks_and_nodes = ox.utils_graph.graph_from_gdfs(all_pts, streets_and_sidewalks)
just_sidewalks_and_nodes = ox.utils_graph.graph_from_gdfs(all_pts, just_sidewalks)

In [ ]:
ox.io.save_graph_shapefile(streets_and_sidewalks_and_nodes, "boston_sidewalks_streets_init")
ox.io.save_graph_shapefile(just_sidewalks_and_nodes, "boston_just_sidewalks_init")